## <span style="color:purple">**Configurações iniciais**</span>

In [1]:
source("R/settings.R")

Loading required package: pacman



In [2]:
# Parâmetros
year <- 2024
month <- 3

In [3]:
S3_PATH <- "s3://ai-analytics-sotq-framework/catho/new_data/"

target_dates_six_months_yoy <- report_dates(year,month,MoM = 5,YoY = 1)
target_dates_six_months <- report_dates(year,month,MoM = 5,YoY = 0)
target_dates_three_months_yoy <- report_dates(year,month,MoM = 2,YoY = 1)
target_dates_three_months <- report_dates(year,month,MoM = 2,YoY = 0)

contact_emps_blacklist <- c(1383457,1382097)

all_channels_cbu <- c("Busca de Vagas","Sugestão de Vagas","Sugestão Delivery", "Aviso de Vagas","Vagas Similares", "Auto-Apply")
all_channels_hbu <- c("Recrutamento Perfeito","Vagas Gratis","Triagem")
all_channels <- c(all_channels_cbu,all_channels_hbu)

## <span style="color:purple">**Relatório de Applies**</span>

In [4]:
# Applies ---------------------------------------------------------------------
# Lendo dados
cbu_applies <- summarise_data(
  target_dates = target_dates_six_months_yoy,
  type = "applies",
  type_level = "enriched",stat_id = "usr_id",
  groups = c("report_channel","apply_device_channel","subscription","apply_quali","year","month","day"))

walk(.x=all_channels_cbu,.f=generate_applies_plots)

Busca de Vagas ok!
Sugestão de Vagas ok!
Sugestão Delivery ok!
Aviso de Vagas ok!
Vagas Similares ok!
Auto-Apply ok!


## <span style="color:purple">**Relatório de Contatos com Apply**</span>

In [5]:
# Contacts from Applies -------------------------------------------------------
cbu_contatos_applies <- summarise_data(
  target_dates = target_dates_six_months_yoy,
  type = "contacts-applies",
  type_level = "merged",stat_id = "usr_id",
  groups = c("report_channel","subscription","year","month","day"))

walk(.x=all_channels_cbu,.f=generate_contacts_applies_plots)

Busca de Vagas ok!
Sugestão de Vagas ok!
Sugestão Delivery ok!
Aviso de Vagas ok!
Vagas Similares ok!
Auto-Apply ok!


## <span style="color:purple">**Relatório de Contatos**</span>

In [6]:
# Contacts --------------------------------------------------------------------
hbu_contatos <- summarise_data(
  target_dates = target_dates_six_months_yoy,
  type = "contacts",
  type_level = "enriched",stat_id = "usr_id",
  groups = c("origem","subscription","year","month","day"))

walk(.x=all_channels_hbu,.f=generate_contacts_plots)

Recrutamento Perfeito ok!
Vagas Gratis ok!
Triagem ok!


## <span style="color:purple">**Relatório de Saúde dos Serviços**</span>

In [7]:
# Health check ----------------------------------------------------------------
logs <- summarise_data(
  target_dates = target_dates_six_months,
  type = "logs-health-check",
  type_level = "raw",stat_id = "status_label",
  groups = c("channel","status_label","year","month","day"))

walk(.x=all_channels,.f=generate_logs_plots)

Busca de Vagas ok!
Sugestão de Vagas ok!
Sugestão Delivery ok!
Aviso de Vagas ok!
Vagas Similares ok!
Auto-Apply ok!
Recrutamento Perfeito ok!
Vagas Gratis ok!
Triagem ok!


## <span style="color:purple">**Relatório de Buscas**</span>

In [8]:
# Top Keywords ----------------------------------------------------------------
date_aux <- data.frame(dates = target_dates_three_months_yoy) |>
  mutate(period = ifelse(dates %in% target_dates_three_months,"3 Months","YoY"))

cbu_top_keywords <- summarise_data(
  target_dates = target_dates_three_months_yoy,
  type = "job-organic-logged-searches",
  type_level = "enriched",stat_id = "usr_id",
  groups = c("keywords_stem","period"),
  df_join = date_aux,df_join_by = c("search_date"="dates")
)

hbu_top_keywords <- summarise_data(
  target_dates = target_dates_three_months_yoy,
  type = "cv-organic-logged-searches",
  type_level = "enriched",stat_id = "emp_id",
  groups = c("keywords_stem","period"),
  df_join = date_aux,df_join_by = c("search_date"="dates")
)

walk(.x=c("Busca de Vagas","Recrutamento Perfeito"),.f=generate_top_keywords_plots)

Busca de Vagas ok!
Recrutamento Perfeito ok!


## <span style="color:purple">**Compartilhamento de Plots**</span>

In [9]:
# S3 ----------------------------------------------------------------
system("aws s3 cp plots s3://ai-analytics-sotq-framework/catho/new_img/graphics --recursive")